# Import Required Libraries
Import the necessary libraries, including pandas and sqlite3.

In [1]:
import os

db_files = ["databases_uc/deliveroo.db", "databases_uc/takeaway.db", "databases_uc/ubereats.db"]

for file in db_files:
    print(f"{file} exists: {os.path.exists(file)}")


databases_uc/deliveroo.db exists: True
databases_uc/takeaway.db exists: True
databases_uc/ubereats.db exists: True


In [2]:
db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]


print(db_files)


[('databases_uc/deliveroo.db', 'deliveroo'), ('databases_uc/takeaway.db', 'takeaway'), ('databases_uc/ubereats.db', 'ubereats')]


In [3]:
import sqlite3

db_files = [
    "databases_uc/deliveroo.db",
    "databases_uc/takeaway.db",
    "databases_uc/ubereats.db"
]

for db_file in db_files:
    print(f"Inspecting schema for {db_file}...")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables: {tables}")
    
    if ('restaurants',) in tables:
        cursor.execute("SELECT sql FROM sqlite_master WHERE name='restaurants';")
        schema = cursor.fetchone()[0]
        print(f"Schema for 'restaurants':\n{schema}")
    else:
        print("No 'restaurants' table found.")
    
    conn.close()


Inspecting schema for databases_uc/deliveroo.db...
Tables: [('locations',), ('restaurants',), ('categories',), ('menu_items',), ('locations_to_restaurants',)]
Schema for 'restaurants':
CREATE TABLE restaurants(
                      id INTEGER PRIMARY KEY,
                      name TEXT,
                      visited_time,
                      latitude,
                      longitude,
                      menu_id,
                      category TEXT,
                      address TEXT,
                      postal_code,
                      prep_time,
                      delivery_time,
                      delivery_fee,
                      fulfillment_method,
                      min_order,
                      phonenumber,
                      rating,
                      rating_number,
                      uname
                      )
Inspecting schema for databases_uc/takeaway.db...
Tables: [('sqlite_sequence',), ('locations',), ('restaurants',), ('locations_to_resta

# Define Database Connection
Establish a connection to the SQLite database using sqlite3.

In [5]:
import sqlite3
import pandas as pd
import os

# Database files and their identifiers
db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]

# Function to load and clean data
def load_restaurant_data(db_file, db_id):
    """Load restaurant data from each database."""
    if not os.path.exists(db_file):
        print(f"Error: Database file {db_file} does not exist.")
        return pd.DataFrame()  # Return empty DataFrame if file is missing

    conn = sqlite3.connect(db_file)
    try:
        if db_id == "deliveroo":
            query = """
            SELECT 
                name AS restaurant_name, 
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                rating,
                CAST(rating_number AS INTEGER) AS rating_number
            FROM restaurants
            """
        elif db_id == "takeaway":
            query = """
            SELECT 
                name AS restaurant_name, 
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                ratings AS rating,
                CAST(ratingsNumber AS INTEGER) AS rating_number
            FROM restaurants
            """
        elif db_id == "ubereats":
            query = """
            SELECT 
                title AS restaurant_name, 
                CAST(location__longitude AS REAL) AS longitude, 
                CAST(location__latitude AS REAL) AS latitude, 
                rating__rating_value AS rating,
                CAST(rating__review_count AS INTEGER) AS rating_number
            FROM restaurants
            """
        else:
            raise ValueError(f"Unknown database identifier: {db_id}")

        # Load data into a DataFrame
        df = pd.read_sql_query(query, conn)
        df['source'] = db_id  # Add source identifier
        return df

    except sqlite3.OperationalError as e:
        print(f"Error accessing table in {db_id}: {e}")
        return pd.DataFrame()

    finally:
        conn.close()

# Load data from all databases
dataframes = [load_restaurant_data(file, db_id) for file, db_id in db_files]

# Combine data into a single stacked DataFrame
stacked_df = pd.concat(dataframes, ignore_index=True)

# Standardize and clean data
stacked_df['longitude'] = stacked_df['longitude'].round(6)
stacked_df['latitude'] = stacked_df['latitude'].round(6)
stacked_df['rating'] = stacked_df['rating'].fillna(0.0)  # Handle missing ratings

# Clean and process 'rating_number' column
if 'rating_number' in stacked_df.columns:
    stacked_df['rating_number'] = (
        pd.to_numeric(stacked_df['rating_number'], errors='coerce')
        .fillna(0)
        .astype(int)
    )

# Display stacked DataFrame
print(f"Stacked DataFrame:\n{stacked_df.head()}")

# Save to CSV for manual inspection
output_file = "stacked_restaurants_with_clean_rating_number.csv"
stacked_df.to_csv(output_file, index=False)
print(f"Stacked data saved to '{output_file}'.")


Stacked DataFrame:
  restaurant_name  longitude   latitude  rating  rating_number     source
0       Meli Mêly   4.399550  50.855476     4.4            411  deliveroo
1  't Misverstand   4.334645  50.794531     0.0              0  deliveroo
2   Mumbaï Dreams   4.352460  50.792346     0.0              0  deliveroo
3         Gri Gri   4.351862  50.792341     0.0              0  deliveroo
4     Le Nénuphar   4.433540  50.848879     0.0              0  deliveroo
Stacked data saved to 'stacked_restaurants_with_clean_rating_number.csv'.


In [3]:
import os
import sqlite3
import pandas as pd

# Database files with identifiers
db_files = [
    ("databases_uc/deliveroo.db", "deliveroo"),
    ("databases_uc/takeaway.db", "takeaway"),
    ("databases_uc/ubereats.db", "ubereats")
]

# Check if databases exist and list tables
for db_path, db_name in db_files:
    if not os.path.exists(db_path):
        print(f"Error: {db_path} does not exist!")
        continue
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables in the {db_name} database:", tables)
    conn.close()

# Function to load and clean restaurant data
def load_restaurant_data(db_file, db_id):
    """Load restaurant data from each database."""
    if not os.path.exists(db_file):
        print(f"Error: Database file {db_file} does not exist.")
        return pd.DataFrame()  # Return empty DataFrame if the file is missing

    conn = sqlite3.connect(db_file)
    try:
        # Define queries based on database
        if db_id == "deliveroo":
            query = """
            SELECT 
                name AS restaurant_name, 
                address AS address,
                postal_code as postal_code,
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                rating
            FROM restaurants
            """
        elif db_id == "takeaway":
            query = """
            SELECT 
                name AS restaurant_name, 
                address AS address,
                city AS city,
                CAST(longitude AS REAL) AS longitude, 
                CAST(latitude AS REAL) AS latitude, 
                ratings AS rating
            FROM restaurants
            """
        elif db_id == "ubereats":
            query = """
            SELECT 
                title AS restaurant_name, 
                location__address AS address,
                location__city AS city,
                CAST(location__longitude AS REAL) AS longitude, 
                CAST(location__latitude AS REAL) AS latitude, 
                rating__rating_value AS rating
            FROM restaurants
            """
        else:
            raise ValueError("Unknown database identifier.")

        # Load data using pandas
        df = pd.read_sql_query(query, conn)
        if df.empty:
            print(f"Warning: Query for {db_id} returned no data.")
        else:
            # Standardize coordinates
            df['longitude'] = df['longitude'].round(6)
            df['latitude'] = df['latitude'].round(6)
        # Add a source column
        df['source'] = db_id
        return df

    except sqlite3.OperationalError as e:
        print(f"Error accessing table in {db_id}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if table is missing

    finally:
        conn.close()

# Load and clean data from all databases
dataframes = [load_restaurant_data(file, db_id) for file, db_id in db_files]

# Combine all data into a single stacked DataFrame
stacked_df = pd.concat(dataframes, ignore_index=True)

# Handle missing values
stacked_df['city'] = stacked_df['city'].fillna("Unknown")
stacked_df['address'] = stacked_df['address'].fillna("Unknown")
stacked_df['rating'] = stacked_df['rating'].fillna(0.0)

# Save the stacked DataFrame for inspection
output_file = "stacked_restaurants_with_full_geolocation.csv"
stacked_df.to_csv(output_file, index=False)
print(f"Stacked data with geolocation saved to '{output_file}'.")

# Check for duplicate entries based on geolocation (optional)
duplicates = stacked_df[stacked_df.duplicated(subset=['restaurant_name', 'longitude', 'latitude'], keep=False)]
if not duplicates.empty:
    print("Duplicate entries found:")
    print(duplicates.head(10))  # Display the first 10 duplicate entries
else:
    print("No duplicate entries found.")


Tables in the deliveroo database: [('locations',), ('restaurants',), ('categories',), ('menu_items',), ('locations_to_restaurants',)]
Tables in the takeaway database: [('sqlite_sequence',), ('locations',), ('restaurants',), ('locations_to_restaurants',), ('menuItems',), ('categories',), ('categories_restaurants',)]
Tables in the ubereats database: [('locations',), ('locations_to_restaurants',), ('menu_items',), ('menu_sections',), ('restaurant_hours_to_section_hours',), ('restaurant_to_categories',), ('restaurant_to_hours',), ('restaurant_to_supported_dining_modes',), ('restaurants',)]
Stacked data with geolocation saved to 'stacked_restaurants_with_full_geolocation.csv'.
Duplicate entries found:
       restaurant_name                                address postal_code  \
13            Bia Mara                      Maalderijstraat 1        2000   
25            Mikuriya                            12 rue Voot        1200   
26     La Nuova Piazza                        Place Meiser 15  

# Read SQL Queries from Files
Read the SQL queries from external files using the open() function.

# Merge DataFrames
Merge the DataFrames on the restaurant name using the merge() function.